In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from utils.dataset_download import *
import pickle
import json
from pathlib import Path, PosixPath
import pandas as pd

In [4]:
# import API credentials
with open('oed_experiments/oed_credentials.json') as f:
    credentials = json.load(f)

In [ ]:
# define lemma
lemma_id = "machine_nn01"

In [ ]:
save_path = Path("./data")
save_path.mkdir(exist_ok=True)

In [ ]:
#query the API and get the json response
sense_json = query_oed(credentials,'word',lemma_id,flags='include_senses=true&include_quotations=true')

# convert the json in a dataframe
senses_df = convert_json_to_dataframe(sense_json)

# convert semantic class ids to labels
#senses_df['semantic_class_ids'] = senses_df['semantic_class_ids'].apply(lambda x: semantic_class_ids_to_descriptions(x,credentials))

In [ ]:
# save the dataframe
# as pickle
senses_df.to_pickle(save_path / f"senses_{lemma_id}.pickle")
# as csv
senses_df.to_csv(save_path / f"senses_{lemma_id}.tsv",sep='\t')

In [ ]:
# open pickle file to avoid calling the API again
with open(save_path / f"senses_{lemma_id}.pickle",'rb') as in_pickle:
    machine_senses_df = pickle.load(in_pickle)

In [ ]:
# get all senses that are siblings and descendants
# of the semantic class of senses listed in previously obtained query 
# TO DOs:
#  - merge all information into one dataframe
# responses = traverse_thesaurus(credentials,machine_senses_df)

In [5]:
# traverse tree or load responses 
# responses = traverse_thesaurus(credentials,machine_senses_df)
with open('./data/tree_traversal.pickle','rb') as in_pickle:
    tt = pickle.load(in_pickle)

In [6]:
quotations = get_quotations_from_thesaurus(credentials,tt)

## Fin.